## Step1 資料庫連線

In [17]:
#把使用的套件都匯入
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
jieba.set_dictionary('E:/text_mining/dict.txt')             #切換至中文繁體字庫
jieba.load_userdict("E:/text_mining/dict_keyw_new.txt")     #加入自建詞庫
jieba.load_userdict("E:/text_mining/dict_cbdic.txt")

#預設就是自己
client = MongoClient()
database = client['testnews']
collection =database['news1']

Building prefix dict from E:\text_mining\dict.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u144803eaa96482ea338b54ad8c7f9634.cache
Loading model cost 1.025 seconds.
Prefix dict has been built succesfully.


## Step2 獲取資料

In [18]:
title=[]
content =[]
#拿資料
for post in collection.find(): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    #print content
    title.append(post['title'])
    #print title
#總文章數量
print "文章數:",len(title)

文章數: 7327


## Step3 計算TF-IDF

In [19]:
#定義計算TF-IDF的函式
def tfIdf(content):
    vectorizer = TfidfVectorizer()
    #print vectorizer
    #help(vectorizer)
    X = vectorizer.fit_transform(content)  # titile 放文本
    #print X
    weight = X.toarray()
    #print weight
    print "特徵值數量:",len(vectorizer.get_feature_names())
    return weight,X,vectorizer
#把上面存好的切詞進行TF-IDF計算，並把結果存進 weight
weight,X,vectorizer = tfIdf(content)

特徵值數量: 125691


## Step4 匯入計算函式

In [20]:
class test(): #宣告類別
    
    @staticmethod #宣告成靜態方法, 靜態方法不需要設定self
    def getTopTfidf(news_id,n):
        indices = np.argsort(X.toarray()[news_id])[::-1] 
        #print indices
        features = vectorizer.get_feature_names()
        #print features
        print title[news_id]
        top_n = n
        top_features = [features[i] for i in indices[:top_n]]
        return top_features,indices

    @staticmethod
    def getTopWeight(news_id,n):
        top_features,indices = test.getTopTfidf(news_id,n) #類別名稱.方法()
        a=0
        for i in top_features:
            print i,weight[news_id][indices[a]]  # weight[2000] 裡面請填入跟上面一樣的文章編號
            a=a+1

    #def getTopWeight(self,news_id,n):  #這是定義實體/非靜態方法, self指的是"類別物件()"本身
    #呼叫實體/非靜態方法, 類別物件().方法()
    #沒有做任何宣告的方法預設會是實體/非靜態方法
    
    #python 還有定義一個類別方法 @classmethod

## Step5 印出某篇文章TOP 特徵詞

In [21]:
##請在這邊輸入要搜尋的文章
#第一個參數輸入文章編號、第二個參數輸入TOP多少的詞        
test.getTopWeight(40,50)   #呼叫靜態方法, 類別名稱.方法()

#test().getTopWeight(0,50) #呼叫實體/非靜態方法, 類別物件().方法() 

MemoryError: 

In [58]:
#正常狀況的字串長度

word1 = "HAHA"
len(word1)

4

In [23]:
#魔術方法, python內建的方法, __方法__()都是這一類的
class kuso:
    def __init__(self, object):
        self.text = object
    def __len__(self):  #override python內建的方法__len__():
        return 999999
    
    def __eq__(self, object):
        return 1 == 2
    
word1 = kuso("HAHA")   #只要塞入這個類別...
print len(word1)       #len()就被玩壞了...不管檢查啥長度都傳回999999

li = kuso([1,2,3,4,5])
print len(li)

#a = kuso(1)
#b = kuso(1)
#a == b  #kuso(1)物件不等於kuso(1)物件? 因為__eq__()被改了

AttributeError: kuso instance has no attribute '__len__'